In [49]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGSMITH_TRACING"] = os.getenv("LANGSMITH_TRACING")
os.environ["LANGSMITH_ENDPOINT"] = os.getenv("LANGSMITH_ENDPOINT")
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGSMITH_PROJECT"] = os.getenv("LANGSMITH_PROJECT")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")

In [50]:
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings 
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0) 

In [51]:
llm.invoke("Can cars fly?").content

'As of now, traditional cars do not have the capability to fly. However, there are several prototypes and concepts for flying cars that have been developed by various companies and inventors. These vehicles, often referred to as "flying cars" or "vertical takeoff and landing (VTOL) aircraft," are designed to combine the features of a car and an aircraft, allowing them to drive on roads and take off or land vertically.\n\nSome of these flying car prototypes have undergone testing, and a few have received regulatory approval for limited use. However, widespread commercial use of flying cars is still in the developmental stage, facing challenges such as safety regulations, air traffic management, and infrastructure requirements.\n\nIn summary, while flying cars are a concept that is being explored, they are not yet a common reality.'

In [52]:
class Chatbot():
    def __init__(self, system=""):
        self.system = system
        self.message = []
        if self.system:
            self.message.append({"role":"system", "content":system})

    def __call__(self,message):
        self.message.append({"role":"user", "content":message})
        result=self.execute()
        self.message.append({"role":"assistant", "content":result})
        return result

    def execute(self):
        llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
        result = llm.invoke(self.message)
        return result.content

In [53]:
bot=Chatbot(system="you are helpful assistant")

In [54]:
bot("Hi how are you")

"I'm just a computer program, so I don't have feelings, but I'm here and ready to help you! How can I assist you today?"

In [55]:
bot.message

[{'role': 'system', 'content': 'you are helpful assistant'},
 {'role': 'user', 'content': 'Hi how are you'},
 {'role': 'assistant',
  'content': "I'm just a computer program, so I don't have feelings, but I'm here and ready to help you! How can I assist you today?"}]

In [56]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop your output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.


Your available actions are:
calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point
syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

simon_blog_search:
e.g. simon_blog_search: Python
Search Simon's blog for that term

Example session:
Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:
Observation: France is a country. The capital is Paris.

You then output:
Answer: The capital of France is Paris

Please Note: if you get basic conversation questions like "hi","hello","how are you?",\n
you have to answer "hi","hello","i am good".
""".strip()

In [57]:
import re
action_re = re.compile('^Action: (\w+): (.*)')

In [58]:
import httpx

def wikipedia(query):
    try:
        response = httpx.get("https://en.wikipedia.org/w/api.php", params={
            "action": "query",
            "list": "search",
            "srsearch": query,
            "format": "json"
        }, timeout=10.0)
        
        response.raise_for_status()  # Raise an error for bad status codes
        
        data = response.json()
        
        # Check if we got valid results
        if "query" in data and "search" in data["query"] and len(data["query"]["search"]) > 0:
            return data["query"]["search"][0]["snippet"]
        else:
            return f"No Wikipedia results found for '{query}'"
            
    except httpx.TimeoutException:
        return f"Wikipedia search timed out for '{query}'"
    except httpx.HTTPError as e:
        return f"Wikipedia API error: {str(e)}"
    except Exception as e:
        return f"Error searching Wikipedia: {str(e)}"

In [59]:
def simon_blog_search(query):
    try:
        response = httpx.get("https://datasette.simonwillison.net/simonwillisonblog.json", params={
            "sql": """
            select
              blog_entry.title || ': ' || substr(html_strip_tags(blog_entry.body), 0, 1000) as text,
              blog_entry.created
            from
              blog_entry join blog_entry_fts on blog_entry.rowid = blog_entry_fts.rowid
            where
              blog_entry_fts match escape_fts(:q)
            order by
              blog_entry_fts.rank
            limit
              1
            """.strip(),
            "_shape": "array",
            "q": query,
        }, timeout=10.0)
        
        response.raise_for_status()
        
        data = response.json()
        
        if data and len(data) > 0 and "text" in data[0]:
            return data[0]["text"]
        else:
            return f"No blog posts found for '{query}'"
            
    except httpx.TimeoutException:
        return f"Blog search timed out for '{query}'"
    except httpx.HTTPError as e:
        return f"Blog API error: {str(e)}"
    except Exception as e:
        return f"Error searching blog: {str(e)}"

In [60]:
def calculate(number):
    return eval(number)

In [61]:
calculate("2+2")

4

In [62]:
known_actions = {
    "wikipedia": wikipedia,
    "calculate": calculate,
    "simon_blog_search": simon_blog_search
}

In [63]:
def query(question, max_turns=5):
    i = 0
    bot = Chatbot(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = f"Observation: {observation}"
        else:
            return result

In [64]:
bot = Chatbot(prompt)

In [65]:
bot("hi how are you?")

'hi, i am good'

In [66]:
bot("What is your job?")

'Thought: I should explain my purpose and function. \nAction: wikipedia: AI assistant\nPAUSE'

In [67]:
next_prompt="Tell me about quantum computing from wikipedia"

In [68]:
next_prompt="Tell me about the current gdp of India from wikipedia"

In [69]:
result = bot(next_prompt)

In [70]:
actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]

In [71]:
actions

[<re.Match object; span=(0, 31), match='Action: wikipedia: GDP of India'>]

In [72]:
action, action_input = actions[0].groups()

In [73]:
action

'wikipedia'

In [74]:
action_input

'GDP of India'

In [75]:
query("Hi")

hi


'hi'

In [76]:
query("How are you")

hi, hello, i am good


'hi, hello, i am good'

In [77]:
query("Fifteen * twenty five")

Thought: I need to perform a multiplication calculation to find the product of fifteen and twenty-five. 
Action: calculate: 15 * 25
PAUSE
 -- running calculate 15 * 25
Observation: 375
Answer: Fifteen multiplied by twenty-five is 375.


'Answer: Fifteen multiplied by twenty-five is 375.'

In [78]:
query("Has Simon written about AI?")

Thought: I need to search Simon's blog to see if he has written about AI. 
Action: simon_blog_search: AI
PAUSE
 -- running simon_blog_search AI
Observation: Talking AI and jobs with Natasha Zouves for News Nation: I was interviewed by News Nation's Natasha Zouves about the very complicated topic of how we should think about AI in terms of threatening our jobs and careers. I previously talked with Natasha two years ago about Microsoft Bing.

I'll be honest: I was nervous about this one. I'm not an economist and I didn't feel confident talking about this topic!

I do find the challenge of making recent advances in AI and LLMs accessible to a general audience absolutely fascinating though, so I took the risk and agreed to the interview.

I think it came out very well. The full hour long video is now available on the News Nation YouTube channel, or as an audio podcast on iTunes or on Spotify.

 

I made my own transcript of the video (using MacWhisper) and fed it into the new Claude Opus 4

'Answer: Yes, Simon has written about AI, specifically discussing its impact on jobs in an interview with Natasha Zouves for News Nation.'

In [79]:
print("Yes, Simon has written about AI, specifically discussing its impact on jobs in an interview with Natasha Zouves for News Nation.")

Yes, Simon has written about AI, specifically discussing its impact on jobs in an interview with Natasha Zouves for News Nation.


In [80]:
query("can you tell me capital of USA")

Thought: I should look up the capital of the USA on Wikipedia to get accurate information.  
Action: wikipedia: United States  
PAUSE
 -- running wikipedia United States  
Observation: Wikipedia API error: Client error '403 Forbidden' for url 'https://en.wikipedia.org/w/api.php?action=query&list=search&srsearch=United+States++&format=json'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Thought: It seems there was an error while trying to access Wikipedia. I can try to find the information through another method.  
Action: wikipedia: USA  
PAUSE
 -- running wikipedia USA  
Observation: Wikipedia API error: Client error '403 Forbidden' for url 'https://en.wikipedia.org/w/api.php?action=query&list=search&srsearch=USA++&format=json'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Thought: Since I cannot access Wikipedia, I will rely on my knowledge to answer the question about the capital of the USA.  
Actio

'Thought: Since I cannot access Wikipedia, I will rely on my knowledge to answer the question about the capital of the USA.  \nAction: None needed, I will provide the answer based on what I know.  \nPAUSE\n\nAnswer: The capital of the USA is Washington, D.C.'